# Ejemplo en Python utilizando el clasificador semi-supervisado en datos reales.

Puede pensar que el Auto-Entrenamiento implica algo de magia o utiliza un enfoque muy complejo. Sin embargo, en realidad, la idea detrás de la autoformación es muy sencilla y puede explicarse mediante los siguientes pasos:

* Primero, recopilamos todos los datos etiquetados y no etiquetados, pero solo usamos observaciones etiquetadas para entrenar nuestro primer modelo supervisado.

* Luego usamos este modelo para predecir la clase de datos no etiquetados.

* En el tercer paso, seleccionamos observaciones que satisfacen nuestros criterios predefinidos (p. ej., la probabilidad de predicción es >90 % o pertenece al top 10 de observaciones con las probabilidades de predicción más altas) y combinamos estas pseudoetiquetas con datos etiquetados.

* Repetimos el proceso entrenando un nuevo modelo supervisado usando observaciones con etiquetas y pseudo-etiquetas . Luego volvemos a hacer predicciones y añadimos las observaciones recién seleccionadas al grupo pseudoetiquetado.

* Repetimos estos pasos hasta que terminamos de etiquetar todos los datos, ninguna observación adicional sin etiquetar satisface nuestros criterios de pseudoetiquetado o alcanzamos el número máximo de iteraciones especificado.

## Configuración
Usaremos los siguientes datos y bibliotecas:

In [1]:
# Data manipulation
import pandas as pd

# Visualization
import plotly.express as px

# Sklearn
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.svm import SVC # for Support Vector Classification baseline model
from sklearn.semi_supervised import SelfTrainingClassifier # for Semi-Supervised learning
from sklearn.metrics import classification_report # for model evaluation metrics

A continuación, descargamos y usamos los datos de la campaña de marketing (fuente: [Kaggle](https://www.kaggle.com/datasets/rodsaldanha/arketing-campaign)). Limitamos la ingestión de archivos a unas pocas columnas críticas, ya que solo usaremos dos funciones para entrenar nuestro modelo de ejemplo.

In [2]:
# Read in data
df = pd.read_csv('marketing_campaign.csv', 
                 encoding='utf-8', delimiter=';',
                 usecols=['ID', 'Year_Birth', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'MntWines', 'MntMeatProducts']
                )

# Create a flag to denote whether the person has any dependants at home (either kids or teens)
df['Dependents_Flag']=df.apply(lambda x: 1 if x['Kidhome']+x['Teenhome']>0 else 0, axis=1)

# Print dataframe
print("Y así es como se ven los datos:")
df

Y así es como se ven los datos:


,ID,Year_Birth,Marital_Status,Income,Kidhome,Teenhome,MntWines,MntMeatProducts,Dependents_Flag
0,5524,1957,Single,58138.0,0,0,635,546,0
1,2174,1954,Single,46344.0,1,1,11,6,1
2,4141,1965,Together,71613.0,0,0,426,127,0
3,6182,1984,Together,26646.0,1,0,11,20,1
4,5324,1981,Married,58293.0,1,0,173,118,1
...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Married,61223.0,0,1,709,182,1
2236,4001,1946,Together,64014.0,2,1,406,30,1
2237,7270,1981,Divorced,56981.0,0,0,908,217,0
2238,8235,1956,Together,69245.0,0,1,428,214,1


Como puede ver, también hemos derivado un 'Dependents_Flag', que usaremos como objetivo de predicción. En otras palabras, intentaremos predecir si nuestro comprador del supermercado tiene dependientes (niños/adolescentes) en casa o no.

Necesitamos hacer un par de cosas más antes de comenzar a entrenar modelos. Dado que nuestro objetivo es evaluar el rendimiento de un clasificador de autoformación, una técnica semisupervisada, dividiremos los datos según la configuración a continuación.

Los datos de prueba se utilizarán para evaluar el rendimiento del modelo, mientras que los datos etiquetados y no etiquetados se utilizarán para entrenar nuestros modelos.

### Entonces, dividamos los datos en muestras de entrenamiento e imprimamos formas para verificar que el tamaño sea correcto:

In [3]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=0)
print('Size of train dataframe: ', df_train.shape[0])
print('Size of test dataframe: ', df_test.shape[0])

Size of train dataframe:  1680
Size of test dataframe:  560


Ahora enmascaremos el 95% de las etiquetas dentro de los datos de entrenamiento y creemos una variable de destino que use '-1' para indicar datos sin etiquetar (enmascarados):

In [5]:
# Create a flag for label masking
df_train['Random_Mask'] = True
df_train.loc[df_train.sample(frac=0.05, random_state=0).index, 'Random_Mask'] = False

# Create a new target colum with labels. 
# The 1's and 0's are original labels and -1 represents unlabeled (masked) data
df_train['Dependents_Target']=df_train.apply(lambda x: x['Dependents_Flag'] 
                                             if x['Random_Mask']==False else -1, axis=1)

# Show target value distribution
print('Target Value Distribution:')
print(df_train['Dependents_Target'].value_counts())

Target Value Distribution:
-1    1596
 1      58
 0      26
Name: Dependents_Target, dtype: int64


Finalmente, representemos los datos de entrenamiento en un diagrama de dispersión 2D para ver cómo se distribuyen las observaciones.

In [6]:
# Create a scatter plot
fig = px.scatter(df_train, x='MntMeatProducts', y='MntWines', opacity=1
                 , color=df_train['Dependents_Target'].astype(str),
                 color_discrete_sequence=['lightgrey', 'red', 'blue'],
                )

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='white', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='white', 
                 showline=True, linewidth=1, linecolor='white')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='white', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='white', 
                 showline=True, linewidth=1, linecolor='white')

# Set figure title
fig.update_layout(title_text="Marketing Campaign Training Data - Labeled vs. Unlabeled")

# Update marker size
fig.update_traces(marker=dict(size=5))

fig.show()

Como puede ver, utilizaremos 'MntMeatProducts' (gasto anual del comprador en productos cárnicos) y 'MntWines' (gasto anual del comprador en vino) como dos características para predecir si el comprador tiene dependientes en casa.

# Entrenamiento del modelo

Ahora que los datos están listos, entrenaremos un modelo de clasificación de máquina de vectores de soporte (CSV) supervisado en datos etiquetados para establecer un punto de referencia de rendimiento del modelo. Nos permitirá juzgar si un enfoque Semi-Supervisado de un paso posterior es mejor o peor que un modelo Supervisado estándar.

In [7]:
########## Step 1 - Data Prep ########## 
# Select only records with known labels
df_train_labeled=df_train[df_train['Dependents_Target']!=-1]

# Select data for modeling 
X_baseline=df_train_labeled[['MntMeatProducts', 'MntWines']]
y_baseline=df_train_labeled['Dependents_Target'].values

# Put test data into an array
X_test=df_test[['MntMeatProducts', 'MntWines']]
y_test=df_test['Dependents_Flag'].values


########## Step 2 - Model Fitting ########## 
# Specify SVC model parameters
model = SVC(kernel='rbf', 
            probability=True, 
            C=1.0, # default = 1.0
            gamma='scale', # default = 'scale'
            random_state=0
           )

# Fit the model
clf = model.fit(X_baseline, y_baseline)


########## Step 3 - Model Evaluation ########## 
# Use score method to get accuracy of the model
print('---------- SVC Baseline Model - Evaluation on Test Data ----------')
accuracy_score_B = model.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_B)
# Look at classification report to evaluate the model
print(classification_report(y_test, model.predict(X_test)))

---------- SVC Baseline Model - Evaluation on Test Data ----------
Accuracy Score:  0.8285714285714286
              precision    recall  f1-score   support

           0       0.77      0.54      0.63       154
           1       0.84      0.94      0.89       406

    accuracy                           0.83       560
   macro avg       0.81      0.74      0.76       560
weighted avg       0.82      0.83      0.82       560



Los resultados de un modelo (CSV) supervisado ya son bastante buenos, con una precisión del 82,85 %. Tenga en cuenta que el puntaje f1 es más alto para la etiqueta = 1 (comprador con dependientes) debido al desequilibrio de clase.

Ahora sigamos un enfoque semisupervisado con el clasificador de autoentrenamiento de Sklearn mientras usamos el mismo modelo (CSV) como estimador base. 

In [8]:
########## Step 1 - Data Prep ########## 
# Select data for modeling - we are including masked (-1) labels this time
X_train=df_train[['MntMeatProducts', 'MntWines']]
y_train=df_train['Dependents_Target'].values


########## Step 2 - Model Fitting ########## 
# Specify SVC model parameters
model_svc = SVC(kernel='rbf', 
                probability=True, 
                C=1.0, # default = 1.0
                gamma='scale', # default = 'scale',
                random_state=0
               )

# Specify Self-Training model parameters
self_training_model = SelfTrainingClassifier(base_estimator=model_svc, 
                                             threshold=0.7,
                                             criterion='threshold', 
                                             max_iter=100, 
                                             verbose=True
                                            )

# Fit the model
clf_ST = self_training_model.fit(X_train, y_train)


########## Step 3 - Model Evaluation ########## 
print('')
print('---------- Self Training Model - Summary ----------')
print('Base Estimator: ', clf_ST.base_estimator_)
print('Classes: ', clf_ST.classes_)
print('Transduction Labels: ', clf_ST.transduction_)
#print('Iteration When Sample Was Labeled: ', clf_ST.labeled_iter_)
print('Number of Features: ', clf_ST.n_features_in_)
print('Feature Names: ', clf_ST.feature_names_in_)
print('Number of Iterations: ', clf_ST.n_iter_)
print('Termination Condition: ', clf_ST.termination_condition_)
print('')

print('---------- Self Training Model - Evaluation on Test Data ----------')
accuracy_score_ST = clf_ST.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_ST)
# Look at classification report to evaluate the model
print(classification_report(y_test, clf_ST.predict(X_test)))

End of iteration 1, added 1432 new labels.
End of iteration 2, added 127 new labels.
End of iteration 3, added 16 new labels.
End of iteration 4, added 3 new labels.

---------- Self Training Model - Summary ----------
Base Estimator:  SVC(probability=True, random_state=0)
Classes:  [0 1]
Transduction Labels:  [0 1 1 ... 1 1 0]
Number of Features:  2
Feature Names:  ['MntMeatProducts' 'MntWines']
Number of Iterations:  5
Termination Condition:  no_change

---------- Self Training Model - Evaluation on Test Data ----------
Accuracy Score:  0.8357142857142857
              precision    recall  f1-score   support

           0       0.80      0.54      0.64       154
           1       0.84      0.95      0.89       406

    accuracy                           0.84       560
   macro avg       0.82      0.74      0.77       560
weighted avg       0.83      0.84      0.82       560



¡Y los resultados están listos! Hemos mejorado el rendimiento del modelo, aunque solo ligeramente con una precisión del 83,57 %. La puntuación F1 también es marginalmente mejor para la etiqueta = 0, impulsada por una precisión mejorada.

Como se mencionó anteriormente en el artículo, podemos elegir cómo seleccionar pseudoetiquetas para el entrenamiento. Podemos basarnos en las mejores predicciones k_best o especificar un umbral de probabilidad específico.

Esta vez, hemos utilizado un umbral de probabilidad de 0,7. Significa que cualquier observación con una probabilidad de clase de 0,7 o superior se agregará al grupo de datos pseudoetiquetados y se usará para entrenar el modelo en la siguiente iteración.

## Referencia

* Dobilas, S. (2021, December 5). Self-Training Classifier: How to Make Any Algorithm Behave Like a Semi-Supervised One | by Saul Dobilas | Towards Data Science. Towardsdatascience. https://towardsdatascience.com/self-training-classifier-how-to-make-any-algorithm-behave-like-a-semi-supervised-one-2958e7b54ab7